In [1]:
from imports import *

/tmp/ipykernel_1241/3918073063.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from imports import *
/home/aarti/.local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in GPT4AllEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
USER_AGENT environment variable not set, consider setting it to identify your requests.


# Data Basic View

In [2]:
import requests
response = requests.get("https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/OME/METADATA.ome.xml")
data = response.text
ome_xml = ome_types.from_xml(response.text.replace("Â",""))
channel_names = [c.name for c in ome_xml.images[0].pixels.channels]
channel_dict = {name: idx for idx, name in enumerate(channel_names)}

In [3]:
df = pd.read_csv("/content/drive/MyDrive/finaloutput.csv", index_col=0)
df["initial_sets"] = "test"

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/finaloutput.csv'

In [ ]:
polygons = np.array(df["vertices"].apply(lambda row: np.array(ast.literal_eval(row)).tolist()).values.tolist()).astype(np.int16)

for i in range(len(polygons)):
    # Swap ordering of vertices 3 and 4
    temp = list(polygons[i, 3, :])
    polygons[i, 3, :] = polygons[i, 2, :]
    polygons[i, 2, :] = temp
centers = np.array(df["center"].apply(lambda row: np.array(ast.literal_eval(row)).tolist()).values.tolist()).astype(np.int16)

In [ ]:
adata = AnnData(obs=df, obsm={"polygons": polygons, "centers": centers})
adata.write_zarr("my_adata.zarr")

In [6]:
class BrickOpacityPlugin(VitesscePlugin):
    def on_config_change(self, new_config):
        update_needed = False
        original_uid = new_config.get("uid", "")
        updated_uid = original_uid

        if "coordinationSpace" in new_config and "obsSetSelection" in new_config["coordinationSpace"]:
            obs_set_selection = new_config["coordinationSpace"]["obsSetSelection"]
            selection_list = obs_set_selection.get("A") or []

            if len(selection_list) == 0:
                if new_config["coordinationSpace"]["spatialChannelOpacity"].get("init_bv_obsSegmentations_0", None) != 0:
                    new_config["coordinationSpace"]["spatialChannelOpacity"]["init_bv_obsSegmentations_0"] = 0
                    update_needed = True
                updated_uid = f"with_query_empty_{uuid.uuid4().hex}"

            elif "My Selections" in [sel[0] for sel in selection_list]:
                if new_config["coordinationSpace"]["spatialChannelOpacity"].get("init_bv_obsSegmentations_0", None) != 1.0:
                    new_config["coordinationSpace"]["spatialChannelOpacity"]["init_bv_obsSegmentations_0"] = 1.0
                    update_needed = True
                updated_uid = f"with_query_{str(selection_list)}"

        if update_needed and updated_uid != original_uid:
            return {**new_config, "uid": updated_uid}
        else:
            return new_config

In [8]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
image_dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"

).add_object(
    AnnDataWrapper(
        adata_store="my_adata.zarr",
        obs_locations_path="obsm/centers",
        obs_segmentations_path="obsm/polygons",
        obs_set_paths=["obs/initial_sets"],
        obs_set_names=["Initial sets"],
    )
)
obs_set_selection_scope, obs_set_color_scope, additional_obs_sets_scope = vc.add_coordination("obsSetSelection", "obsSetColor", "additionalObsSets")
obs_set_selection_scope.set_value(None)
obs_set_color_scope.set_value(None)
additional_obs_sets_scope.set_value(None)

spatial = vc.add_view("spatialBeta", dataset=image_dataset)
lc = vc.add_view("layerControllerBeta", dataset=image_dataset)
obs = vc.add_view("obsSets", dataset=image_dataset)

vc.link_views_by_dict([spatial, lc], {
    "spatialTargetT": 0,
    "spatialZoom": -3.55,
    "spatialTargetX": 6478,
    "spatialTargetY": 2215,
    "spatialTargetZ": 100,
    "spatialRenderingMode": "2D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 45,
            "spatialChannelColor": [255,0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
              "spatialChannelWindow": [0, 100],
          }

          ])
      }

    ]),
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.link_views_by_dict([spatial, lc], {
    "segmentationLayer": CL([
      {
        "spatialLayerVisible": True,
        "spatialLayerOpacity": 1.0,
        "segmentationChannel": CL([
          {
            "obsType": 'cell',
            "spatialTargetC": 0,
            "spatialChannelColor": [255, 255, 255],
            "spatialChannelOpacity": 0, # Note: opacity setting seems buggy (only becomes transparent with a very small value)
            "featureType": 'gene',
            "featureValueType": 'expression',
            "spatialChannelVisible": True,
            "obsColorEncoding": 'spatialChannelColor',
            "spatialSegmentationFilled": False,
            "spatialSegmentationStrokeWidth": 25,
            "obsHighlight": None,
            "obsSetSelection": obs_set_selection_scope,
            "obsSetColor": obs_set_color_scope,
            "additionalObsSets": additional_obs_sets_scope,
          },
        ])
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "obsSegmentations"))

obs.use_coordination(obs_set_selection_scope)
obs.use_coordination(obs_set_color_scope)
obs.use_coordination(additional_obs_sets_scope)

vc.layout(spatial | lc);
vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False, plugins=[BrickOpacityPlugin()])
vw

# Open AI Key + Prompt

In [9]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model="gpt-4o")

In [11]:
promptexample =  """
You are a helpful assistant. Always provide your answer in exactly two paragraphs and no more. Do not add in any extra lines.

First, always provide a brief description of the marker(s) or region asked about in their query and what they do,
and second, if the user just asks to see or show a certain number of markers, provide a code block following the exact structure of the example provided below.
Below are guidelines you must always follow.

1. Place the marker number specified in the prompt in the example structure provided below. Make sure to place the correct marker number as specified in channel_dict in SpatialTargetC.

2. Always provide the full structure provided below, do not skip out or show only specific parts. Ensure the code block is complete, in valid syntax, and without comments or markdown.

3. If no colors are specified, please choose any color but have it be different for each unless specified otherwise.

4. When a vague or odd question is asked think more about it in context of the tissue and the channel markers provided and answer the question. Please rememeber the user can be a student who
may not know all the terminology.

5. When asked for a region with a combination of low and high markers please show both.

6. Do not display or choose any markers outside of the provided to you in the channel_dict. Only provide description and code block/region for the ones within the dict. If the related markers do not exist in
the list, simply say there aren't any of those markers available to view.

7. If you are provided with a list of coordinates and asked a query on what's happening here ask the user if they want to view for the markers already displayed in the view or across all markers available in the tissue.
Then, use the tool find_channel_means_at_coords which will provide you the average channel mean at those regions, the overall channel mean across the tissue, max of that channel and min. Interpret those values and provide the user with a description with what is happening there
along with a selection of the markers you are talking about. Follow the same instructions earlier where you respond in two paragraphs. Do not display anymore than 6 important markers but feel free to talk about more in the description.

9. For all concnetration region related queries, please return the final_coords in the first paragraph description response as such: brick_ids = [], where the ids which are y.x format are inside the array. Make sure you do this, it's important!
Please check your response to make sure you have included the brick_ids in the response.
Structure example (always start with ```python):

"spatialTargetT": 0,
"spatialZoom": -3.55,
"spatialTargetX": 6478,
"spatialTargetY": 2215,
"spatialTargetZ": 100,
"imageLayer": [
      {{
        "spatialTargetC": 4,
        "spatialChannelColor": [255,0,0],
        "spatialChannelVisible": True,
        "spatialChannelOpacity": 1.0,
        "spatialChannelWindow": [0, 100]
      }},
      {{
        "spatialTargetC": 10,
        "spatialChannelColor": [0,255,0],
        "spatialChannelVisible": True,
        "spatialChannelOpacity": 1.0,
        "spatialChannelWindow": [0, 100]
      }},
      {{
        "spatialTargetC": 20,
        "spatialChannelColor": [0,0,255],
        "spatialChannelVisible": True,
        "spatialChannelOpacity": 1.0,
        "spatialChannelWindow": [0, 100]
      }}
]



channel_dict:

['Hoechst': 66,
 "5'hmC": 1,
 'MX1': 2,
 'MART1': 3,
 'CD3E (do not use)': 5,
 'MHC-I': 6,
 'SOX10': 7,
 'S100B': 9,
 'MITF': 10,
 'GranzymeB (do not use)': 11,
 'pan-cytokeratin': 13,
 'lamin-ABC': 14,
 'PDL1': 15,
 'PD1 (do not use)': 17,
 'S100A': 18,
 'CD31': 19,
 'CD206': 21,
 'pMLC2': 22,
 'CD11b (do not use)': 23,
 'CD4': 25,
 'LAG3': 26,
 'CD20': 27,
 'PRAME': 29,
 'CD163': 30,
 'IRF1': 31,
 'B-catenin': 33,
 'CD3E': 34,
 'CD8a': 35,
 'CD11b': 37,
 'FOXP3': 38,
 'PD1': 39,
 'Ki67': 41,
 'CD11c': 42,
 'COX-IV': 43,
 'LysozymeC': 45,
 'SOX9': 46,
 'PMEL': 47,
 'CD103': 48,
 'CyclinD1': 50,
 'BAF1': 51,
 'B-actin': 53,
 'Mast cell tryptase': 54,
 'CD15': 55,
 'Podoplanin': 56,
 'B-tubulin': 58,
 'Catalase': 59,
 'y-H2AX': 60,
 'E-cadherin': 62,
 'Vimentin': 63,
 'Neurofilament L (do not use)': 64,
 'GranzymeB': 65,
 'MHC-II': 67,
 'H3K27me3': 68,
 'Collagen (SHG)': 69]

"""


# Tool Calling Agent + Combined Functions

In [14]:
tools = [find_high_interaction_regions, find_low_interaction_regions, find_high_low_interaction_regions, find_channel_means_at_coords]
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", promptexample),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)


# Vitessce Chat Widget PlugIn

In [ ]:
PLUGIN_ESM = transform("""
function createPlugins(utilsForPlugins) {
    const {
        React,
        PluginFileType,
        PluginViewType,
        PluginCoordinationType,
        PluginJointFileType,
        z,
        useCoordination,
        invokeCommand
    } = utilsForPlugins;

    const CSS = `
        .chat {
            overflow-y: scroll;
        }
    `;

    function ChatView(props) {
        const [nextMessage, setNextMessage] = React.useState('');
        const [isLoading, setIsLoading] = React.useState(false);
        const [chatHistory, setChatHistory] = React.useState([]);
        const inputRef = React.useRef(null); // Reference for the input field

        async function handleClick() {
            if (!nextMessage.trim()) return; // Prevent sending empty messages
            setChatHistory(prev => ([
                ...prev,
                { user: 'You', text: nextMessage },
            ]));
            setIsLoading(true);

            try {
                const [chatReceiveValue, chatReceiveBuffers] = await invokeCommand("chat_send", nextMessage, []);
                setChatHistory(prev => ([
                    ...prev,
                    { user: 'AI', text: chatReceiveValue.text },
                ]));
            } catch (error) {
                console.error("Error during chat interaction:", error);
            }

            setIsLoading(false);
            setNextMessage(''); // Clear input after message sent
            inputRef.current.focus(); // Refocus the input field
        }

        return (
            <>
                <style>{CSS}</style>
                <div className="chat">
                    <p>Chat view</p>
                    <div>
                        {chatHistory.map((message, index) => (
                            <p key={`${index}`}>
                                <b>{message.user}</b>: {message.text}
                            </p>
                        ))}
                    </div>
                    <input
                        type="text"
                        value={nextMessage}
                        onChange={e => setNextMessage(e.target.value)}
                        ref={inputRef} // Assign the ref to the input
                        onKeyDown={(e) => {
                            if (e.key === 'Enter' && !isLoading) {
                                handleClick();
                            }
                        }}
                    />
                    <button onClick={handleClick} enabled={isLoading}>Send message</button>
                </div>
            </>
        );
    }

    const pluginViewTypes = [
        new PluginViewType('chat', ChatView, []),
    ];
    return { pluginViewTypes };
}
export default { createPlugins };



""")

In [ ]:
def send_message(message, buffers):
    global vw
    if vw is None:
        print("Widget not initialized.")
        return

    old_config = copy.deepcopy(vw.config)
    selected_bricks = extract_selected_bricks(old_config)

    if selected_bricks:
        message += f" The user has selected the following bricks: {selected_bricks}."

    descr, output_text = invoke_agent(message)
    brick_ids_to_highlight = extract_brick_ids(output_text)
    new_config = parse_and_merge_config(old_config, output_text)

    if brick_ids_to_highlight:
        new_config = update_config_with_brick_ids(new_config, brick_ids_to_highlight)

    apply_config_to_widget(vw, new_config)

    return {**new_config, "text": descr}, []

class ChatPlugin(VitesscePlugin):
    plugin_esm = PLUGIN_ESM
    commands = {
        "chat_send": send_message,
    }

In [ ]:
vc = VitessceConfig(schema_version="1.0.16", name="BioMedVis Challenge")
image_dataset = vc.add_dataset(name="Blood Vessel", uid="bv").add_file(
    url="https://lsp-public-data.s3.amazonaws.com/yapp-2023-3d-melanoma/Dataset1-LSP13626-melanoma-in-situ/0",
    file_type="image.ome-zarr"

).add_object(
    AnnDataWrapper(
        adata_store="my_adata.zarr",
        obs_locations_path="obsm/centers",
        obs_segmentations_path="obsm/polygons",
        obs_set_paths=["obs/initial_sets"],
        obs_set_names=["Initial sets"],
    )
)
obs_set_selection_scope, obs_set_color_scope, additional_obs_sets_scope = vc.add_coordination("obsSetSelection", "obsSetColor", "additionalObsSets")
obs_set_selection_scope.set_value(None)
obs_set_color_scope.set_value(None)
additional_obs_sets_scope.set_value(None)

spatial = vc.add_view("spatialBeta", dataset=image_dataset)
lc = vc.add_view("layerControllerBeta", dataset=image_dataset)
obs = vc.add_view("obsSets", dataset=image_dataset)
# spatial = vc.add_view("spatialBeta", dataset=dataset)
status = vc.add_view("chat", dataset=image_dataset)
# lc = vc.add_view("layerControllerBeta", dataset=dataset)

vc.link_views_by_dict([spatial, lc], {
    "spatialTargetT": 0,
    "spatialZoom": -4.1,
    "spatialTargetX": 6213,
    "spatialTargetY": 3409,
    "spatialTargetZ": 100,
    "spatialRenderingMode": "2D",
    "imageLayer": CL([
      {
        "spatialTargetResolution": 3,
        "spatialLayerOpacity": 1.0,
        "spatialLayerVisible": True,
        "photometricInterpretation": "BlackIsZero",
        "imageChannel": CL([
          {
            "spatialTargetC": 9,
            "spatialChannelColor": [255, 125,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,9486]
          },
          {
            "spatialTargetC": 19,
            "spatialChannelColor": [0, 0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [888,6882]
          },
          {
            "spatialTargetC": 69,
            "spatialChannelColor": [255, 255, 0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [6,34]
          },
          {
            "spatialTargetC": 21,
            "spatialChannelColor": [255, 0,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [0,6552]
          },
          {
            "spatialTargetC": 29,
            "spatialChannelColor": [255,0,255],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [475,2983]
          },
          {
            "spatialTargetC": 3,
            "spatialChannelColor": [0, 255,0],
            "spatialChannelVisible": True,
            "spatialChannelOpacity": 1.0,
            "spatialChannelWindow": [1287,14825]
          },
        ]),
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "image"))

vc.link_views_by_dict([spatial, lc], {
    "segmentationLayer": CL([
      {
        "spatialLayerVisible": True,
        "spatialLayerOpacity": 1.0,
        "segmentationChannel": CL([
          {
            "obsType": 'cell',
            "spatialTargetC": 0,
            "spatialChannelColor": [255, 255, 255],
            "spatialChannelOpacity": 0, # Note: opacity setting seems buggy (only becomes transparent with a very small value)
            "featureType": 'gene',
            "featureValueType": 'expression',
            "spatialChannelVisible": True,
            "obsColorEncoding": 'spatialChannelColor',
            "spatialSegmentationFilled": False,
            "spatialSegmentationStrokeWidth": 25,
            "obsHighlight": None,
            "obsSetSelection": obs_set_selection_scope,
            "obsSetColor": obs_set_color_scope,
            "additionalObsSets": additional_obs_sets_scope,
          },
        ])
      }
    ])
}, meta=True, scope_prefix=get_initial_coordination_scope_prefix("bv", "obsSegmentations"))

obs.use_coordination(obs_set_selection_scope)
obs.use_coordination(obs_set_color_scope)
obs.use_coordination(additional_obs_sets_scope)

vc.layout((spatial / status ) | (lc / obs));

In [ ]:
vw = vc.widget(js_package_version="3.4.5", remount_on_uid_change=False, plugins=[ChatPlugin(), BrickOpacityPlugin ()])
vw

In [ ]:
vw.config